In [1]:
%%bash
lein uberjar

Retrieving org/clojure/tools.namespace/0.3.1/tools.namespace-0.3.1.jar from central
Retrieving org/clojure/tools.reader/1.3.2/tools.reader-1.3.2.jar from central
Retrieving org/clojure/java.classpath/0.3.0/java.classpath-0.3.0.jar from central
Retrieving clj-time/clj-time/0.14.0/clj-time-0.14.0.jar from clojars
Compiling ppdsp.classifier.base
Compiling ppdsp.classifier.inspectablearf
Compiling ppdsp.classifier.moa-classifier
Compiling ppdsp.classifier.random
Compiling ppdsp.core
Compiling ppdsp.dataset.base
Compiling ppdsp.dataset.csv-dataset
Compiling ppdsp.dataset.moa
Compiling ppdsp.dataset.save-csv
Compiling ppdsp.masking.attack-data
Compiling ppdsp.masking.base
Compiling ppdsp.masking.data_fitting
Compiling ppdsp.masking.evaluation
Compiling ppdsp.masking.optimize
Compiling ppdsp.masking.projection
Compiling ppdsp.masking.single-stage-cumulative-attack
Compiling ppdsp.masking.two-stage-cumulative-attack
Compiling ppdsp.masking.two-stage-independent-attack
Compiling ppdsp.masking.u

In [2]:
 %classpath add jar ../target/jvm/uberjar/ppdsp-0.1.0-SNAPSHOT-standalone.jar
(clojure.lang.Compiler/loadFile "jupyter_helpers.clj")
(require '[clojure.string :as string]
         '[clojure.data.csv :as csv]
         '[clojure.math.numeric-tower :refer [ceil]]
         '[clojure.java.io :as io]
         '[clojure.pprint :refer [pprint print-table]]
         '[jupyter-helpers :refer [save-data load-data display-table 
                                   round-known-record-counts
                                   ;;added by Waruni
                                    read-data run-masking-experiments-cycles  
                                     save-data-append-newline 
                                   validate-datafitting-results
                                   ]]
         '[ppdsp.classifier.moa-classifier :refer [hoeffding-tree ]]    ;;changed
         '[ppdsp.dataset.base :refer [dataset-feature-count dataset-record-count dataset-class-count get-schema]]
         '[ppdsp.dataset.csv-dataset :refer [read-csv-dataset]]
         '[ppdsp.masking.evaluation :refer [flatten-masking-experiment-recoveries
                                           unknown-record-relative-position
                                           add-combined-result
                                           get-cumulative-noise-sigma  calculate-avarage-information-loss-error 
                                            get-attack-count
                                               ]]
         '[ppdsp.masking.data_fitting :refer [kernel-regression-using-fastmath k-AEL-final-from-selected-kernel]] ;retrieve-accuracy-using-fitted-function 
         '[ppdsp.utils :refer [map-vals mean]]
         '[incanter.core ]
         )
         


null

## Experiment Configuration

In [3]:
(def dataset-label "arem")
(def dataset (read-csv-dataset "datasets/arem/arem_all_interleaved.csv"
                                99999999 ;; All records
                                ;; Numeric features
                                ["avg_rss12" "var_rss12" "avg_rss13" "var_rss13" "avg_rss23" "var_rss23"]))


#'beaker_clojure_shell_6229e592-15d5-4bec-ac1f-11ad554a2481/dataset

In [4]:
(def epsilons 0.2)
(def feature-count (dataset-feature-count dataset))
(def record-count (dataset-record-count dataset))
(def window-size 10000)
(def no-of-windows (int (ceil (/ record-count window-size ))) )
(def cycle-sizes [500]) ;;Randomly select the cycle size -->initial cycle size has used to define the cycles
(def attack-count (*(int (/(* window-size 0.05) 4)) no-of-windows )) ;total attack count for all windows. Attack using (window-count * 5%)
;of known I/O pairs.Divided by 4 since 4 I/O pairs are used for one attack
(def independent-noise-sigmas [0.25])
(def cumulative-noise-sigmas (map #(get-cumulative-noise-sigma % record-count) independent-noise-sigmas))
(def class-count (dataset-class-count dataset))

(def base-configuration
    {:dataset dataset
     :projection-feature-counts [feature-count]
     :projection-sigmas [1.0]
     :translations [0]
     :known-record-counts [4] ;;per attack->Total known record count = 2 * attack count 
     :known-record-ranges [1]
     :classifier-fns {:hdt hoeffding-tree} 
     :attempt-count 3
     :attack-count  attack-count
     :threads-per-configuration 2
     :threads-per-evaluation 1
     :seed 1
     :record-count record-count 
     :window-size window-size
     :evaluations [:privacy :accuracy ]
    }
)

(def logistic-cumulative-noise-configuration
    (merge base-configuration
           {:output-file (str "workspace/" dataset-label "/logistic-cumulative-output.edn")
            :output-file-privacy-auc  (str "workspace/" dataset-label "/logistic-cumulative-privacy-auc.csv")
            :attck-results-file  (str "workspace/" dataset-label "/attack-results.edn")
            :output-file-privacy-attacks  (str "workspace/" dataset-label "/logistic-cumulative-privacy-attacks.csv")
            :original-model-description (str "workspace/" dataset-label "/original-model-description.edn")
            :original-probability-file (str "workspace/" dataset-label "/original-AEL.csv")
            :masked-model-file (str "workspace/" dataset-label "/masked-model-description.edn")
            :masked-probability-file (str "workspace/" dataset-label "/masked-AEL.csv")
            :anytime-ael-file (str "workspace/" dataset-label "/anytime-AEL.csv")
            :bp-window-file (str "workspace/" dataset-label "/bp-per-window.csv")
            :maximum-fn-value 1  ;[1]
            :growth-rate-k-initial 0.097; ; will be run for different k values upto 0.1 with tilda= 0.004
            :cumulative-noise-sigmas cumulative-noise-sigmas
            :cycle-sizes cycle-sizes
            :epsilon epsilons
            :attack-strategies [:a-rp :a-rpcn :a-rpcn-1]
            :previous-AUC-file-to-update (str "workspace/" dataset-label "/previous-AUC.csv")
            :previous-upper-bound-file-to-update(str "workspace/" dataset-label "/previous-upper-bound.csv")
            :win-acc-details-file (str "workspace/" dataset-label "/win-acc-details.edn")
            :win-acc-excel-file (str "workspace/" dataset-label "/window-based-acc.csv")
            :average-error-win-file (str "workspace/" dataset-label "/avg-error-per-win.csv")
            }
    )
)



#'beaker_clojure_shell_6229e592-15d5-4bec-ac1f-11ad554a2481/logistic-cumulative-noise-configuration

## Run Experiments

In [5]:
(run-masking-experiments-cycles logistic-cumulative-noise-configuration)

id 20001
Accuracy 0.6993800619938007
id 20002
Accuracy 0.6993450982352647
id 20003
Accuracy 0.6993101379724055
id 20004
Accuracy 0.6992751812046988
id 20005
Accuracy 0.6992402279316206
id 20006
Accuracy 0.6992052781526465
id 20007
Accuracy 0.6991703318672531
id 20008
Accuracy 0.6991353890749163
id 20009
Accuracy 0.6991004497751124
id 20010
Accuracy 0.699065513967318
id 20011
Accuracy 0.6990305816510094
id 20012
Accuracy 0.6989956528256633
id 20013
Accuracy 0.6989607274907566
id 20014
Accuracy 0.6989258056457657
id 20015
Accuracy 0.6988908872901679
id 20016
Accuracy 0.6988559724234401
id 20017
Accuracy 0.6988210610450594
id 20018
Accuracy 0.6987861531545032
id 20019
Accuracy 0.6987512487512488
id 20020
Accuracy 0.6987163478347735
id 20021
Accuracy 0.698681450404555
id 20022
Accuracy 0.6986465564600709
id 20023
Accuracy 0.698611666000799
id 20024
Accuracy 0.6985767790262172
id 20025
Accuracy 0.6985418955358035
id 20026
Accuracy 0.6985070155290358
id 20027
Accuracy 0.6984721390053925
id 2

Accuracy 0.6917029272151899
id 20224
Accuracy 0.6916687268232387
id 20225
Accuracy 0.6916345298131118
id 20226
Accuracy 0.691600336184308
id 20227
Accuracy 0.6915661459363259
id 20228
Accuracy 0.6915319590686638
id 20229
Accuracy 0.6914977755808206
id 20230
Accuracy 0.691463595472295
id 20231
Accuracy 0.691429418742586
id 20232
Accuracy 0.6913952453911926
id 20233
Accuracy 0.6913610754176139
id 20234
Accuracy 0.6913269088213491
id 20235
Accuracy 0.6912927456018976
id 20236
Accuracy 0.6912585857587586
id 20237
Accuracy 0.691224429291432
id 20238
Accuracy 0.691190276199417
id 20239
Accuracy 0.6911561264822134
id 20240
Accuracy 0.6911219801393212
id 20241
Accuracy 0.6910878371702401
id 20242
Accuracy 0.6910536975744702
id 20243
Accuracy 0.6910195613515115
id 20244
Accuracy 0.6909854285008644
id 20245
Accuracy 0.690951299022029
id 20246
Accuracy 0.6909171729145059
id 20247
Accuracy 0.6908830501777953
id 20248
Accuracy 0.6908489308113981
id 20249
Accuracy 0.6908148148148148
id 20250
Accurac

Accuracy 0.6841925070918518
id 20446
Accuracy 0.6841590453367242
id 20447
Accuracy 0.6841255868544601
id 20448
Accuracy 0.6840921316445792
id 20449
Accuracy 0.6840586797066015
id 20450
Accuracy 0.6840252310400469
id 20451
Accuracy 0.6839917856444357
id 20452
Accuracy 0.6839583435192881
id 20453
Accuracy 0.6839249046641244
id 20454
Accuracy 0.6838914690784649
id 20455
Accuracy 0.6838580367618303
id 20456
Accuracy 0.683824607713741
id 20457
Accuracy 0.6837911819337179
id 20458
Accuracy 0.6837577594212816
id 20459
Accuracy 0.6837243401759531
id 20460
Accuracy 0.6836909241972533
id 20461
Accuracy 0.6836575114847034
id 20462
Accuracy 0.6836241020378244
id 20463
Accuracy 0.6835906958561376
id 20464
Accuracy 0.6835572929391645
id 20465
Accuracy 0.6835238932864263
id 20466
Accuracy 0.6834904968974447
id 20467
Accuracy 0.6834571037717413
id 20468
Accuracy 0.6834237139088378
id 20469
Accuracy 0.683390327308256
id 20470
Accuracy 0.6833569439695178
id 20471
Accuracy 0.6833235638921454
id 20472
Acc

id 20668
Accuracy 0.6768106826648604
id 20669
Accuracy 0.67677793904209
id 20670
Accuracy 0.676745198587393
id 20671
Accuracy 0.6767124613003096
id 20672
Accuracy 0.6766797271803802
id 20673
Accuracy 0.6766469962271452
id 20674
Accuracy 0.676614268440145
id 20675
Accuracy 0.6765815438189205
id 20676
Accuracy 0.676548822363012
id 20677
Accuracy 0.6765161040719605
id 20678
Accuracy 0.6764833889453068
id 20679
Accuracy 0.6764506769825919
id 20680
Accuracy 0.6764179681833568
id 20681
Accuracy 0.6763852625471424
id 20682
Accuracy 0.6763525600734903
id 20683
Accuracy 0.6763198607619416
id 20684
Accuracy 0.6762871646120378
id 20685
Accuracy 0.6762544716233201
id 20686
Accuracy 0.6762217817953304
id 20687
Accuracy 0.6761890951276102
id 20688
Accuracy 0.6761564116197013
id 20689
Accuracy 0.6761237312711454
id 20690
Accuracy 0.6760910540814847
id 20691
Accuracy 0.676058380050261
id 20692
Accuracy 0.6760257091770164
id 20693
Accuracy 0.6759930414612931
id 20694
Accuracy 0.6759603769026336
id 2069

Accuracy 0.6696184960030634
id 20891
Accuracy 0.669586444572085
id 20892
Accuracy 0.6695543962092567
id 20893
Accuracy 0.669522350914138
id 20894
Accuracy 0.6694903086862886
id 20895
Accuracy 0.669458269525268
id 20896
Accuracy 0.669426233430636
id 20897
Accuracy 0.6693942004019523
id 20898
Accuracy 0.669362170438777
id 20899
Accuracy 0.6693301435406699
id 20900
Accuracy 0.669298119707191
id 20901
Accuracy 0.6692660989379007
id 20902
Accuracy 0.669234081232359
id 20903
Accuracy 0.6692020665901263
id 20904
Accuracy 0.669170055010763
id 20905
Accuracy 0.6691380464938295
id 20906
Accuracy 0.6691060410388865
id 20907
Accuracy 0.6690740386454945
id 20908
Accuracy 0.6690420393132144
id 20909
Accuracy 0.6690100430416069
id 20910
Accuracy 0.6689780498302329
id 20911
Accuracy 0.6689460596786534
id 20912
Accuracy 0.6689140725864295
id 20913
Accuracy 0.6688820885531223
id 20914
Accuracy 0.6688501075782931
id 20915
Accuracy 0.6688181296615032
id 20916
Accuracy 0.6687861548023138
id 20917
Accuracy 

...

id 34974
Accuracy 0.3999714081486776
id 34975
Accuracy 0.3999599725526075
id 34976
Accuracy 0.3999485376104297
id 34977
Accuracy 0.3999371033220882
id 34978
Accuracy 0.3999256696875268
id 34979
Accuracy 0.3999142367066895
id 34980
Accuracy 0.3999028043795203
id 34981
Accuracy 0.3998913727059631
id 34982
Accuracy 0.3998799416859618
id 34983
Accuracy 0.3998685113194603
id 34984
Accuracy 0.3998570816064027
id 34985
Accuracy 0.399845652546733
id 34986
Accuracy 0.399834224140395
id 34987
Accuracy 0.3998227963873328
id 34988
Accuracy 0.3998113692874904
id 34989
Accuracy 0.3997999428408117
id 34990
Accuracy 0.3997885170472407
id 34991
Accuracy 0.3997770919067215
id 34992
Accuracy 0.3997656674191981
id 34993
Accuracy 0.3997542435846145
id 34994
Accuracy 0.3997428204029147
id 34995
Accuracy 0.3997313978740427
id 34996
Accuracy 0.3997199759979427
id 34997
Accuracy 0.3997085547745585
id 34998
Accuracy 0.3996971342038344
id 34999
Accuracy 0.3996857142857143
id 35000
Accuracy 0.3996742950201423
id 

Accuracy 0.3974486461914368
id 35197
Accuracy 0.3974373543951361
id 35198
Accuracy 0.397426063240433
id 35199
Accuracy 0.3974147727272727
id 35200
Accuracy 0.3974034828556007
id 35201
Accuracy 0.3973921936253622
id 35202
Accuracy 0.3973809050365026
id 35203
Accuracy 0.3973696170889672
id 35204
Accuracy 0.3973583297827013
id 35205
Accuracy 0.3973470431176504
id 35206
Accuracy 0.3973357570937598
id 35207
Accuracy 0.3973244717109748
id 35208
Accuracy 0.3973131869692408
id 35209
Accuracy 0.3973019028685033
id 35210
Accuracy 0.3972906194087075
id 35211
Accuracy 0.3972793365897989
id 35212
Accuracy 0.3972680544117229
id 35213
Accuracy 0.3972567728744249
id 35214
Accuracy 0.3972454919778503
id 35215
Accuracy 0.3972342117219446
id 35216
Accuracy 0.397222932106653
id 35217
Accuracy 0.3972116531319212
id 35218
Accuracy 0.3972003747976944
id 35219
Accuracy 0.3971890971039182
id 35220
Accuracy 0.397177820050538
id 35221
Accuracy 0.3971665436374993
id 35222
Accuracy 0.3971552678647475
id 35223
Accu

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Attacks were performed
Breach probability and accuracy was calculated
Written into the file
"Elapsed time: 397916.4341 msecs"


null

# Accuracy Behavior 

# Privacy Behavior

# Data Fitting Using Privacy and Accuracy Results

In [7]:
;;LOGISTIC NOISE
;(def output-cumulative-logistic (load-data (:output-file logistic-cumulative-noise-configuration))) ;;all results
;(def original-accuracy-cumulative-logistic (-> output-cumulative-logistic :original :accuracy))        ;;original classification results with accuracy

;(def results-cumulative-logistic (-> (filter #(contains? (set cumulative-noise-sigmas) (:cumulative-noise-sigma %)) ;;All results without original accuracy (masking results)
                                   ; (:results output-cumulative-logistic))
                            ;(add-combined-result :score [:a-rp :a-rpcn])
                            ;(add-combined-result :score [:a-rp :a-rpcn-1])
                                   ;  ))

;(def flat-results-cumulative-logistic (->> results-cumulative-logistic
                                 ; flatten-masking-experiment-recoveries
                                 ; (map #(assoc % :unknown-record-relative-position (unknown-record-relative-position %)))  ;;results with privacy
                                       ;    )) ;;results with privacy
                   ;(println flat-results-cumulative-logistic)

                     ;;for wriring new attack details to a file - logistic (relative error of each attacked data record)
;(def epsilon 0.2)
;(def breach-temporary-file (str "workspace/" dataset-label "/breach-temporary.edn"))
      ; (attack-per-cycle flat-results-cumulative-logistic breach-temporary-file epsilon attack-count)
       
       
      

null

In [8]:
;;Read breach probabilities of each run from the temporary file and write it to the final file.
 
;(def breach-probability-file (str "workspace/" dataset-label "/final-breach-probabilities.csv"))
;(save-bp-final breach-temporary-file breach-probability-file (:growth-rate-k logistic-cumulative-noise-configuration))

null

In [7]:
;;;;;;;;;;;;;;;;;;;Read Data from existing csv files 
;(def privacy-breach (to-matrix (load-data (str "workspace/" dataset-label "/final-breach-probabilities.csv"))))
;(def accuracy (load-data (str "workspace/" dataset-label "/masked-AEL.csv")))
(def accuracy-file (str "workspace/" dataset-label "/masked-AEL.csv"))
(def user-threshold-testing [0.6 0.7 0.75 0.8 0.85 0.9 0.95 0.97])
  ;;[0.5 0.55 0.6 0.65 0.7 0.75 0.8 0.85 0.9 0.95] 0.85 0.9 0.92 0.95 0.97
(def datafitting-results-file (str "workspace/" dataset-label "/results-data-fitting.csv"))

;;This is only for the data fitting to select the best kernel
(kernel-regression-using-fastmath (:final-bp-file logistic-cumulative-noise-configuration) accuracy-file user-threshold-testing datafitting-results-file (:output-file-privacy-auc logistic-cumulative-noise-configuration))

;(retrieve-accuracy-using-fitted-function (:final-bp-file logistic-cumulative-noise-configuration) accuracy-file privacy-threshold-by-user final-results-file (:output-file-privacy-auc logistic-cumulative-noise-configuration) variance-smooth-bp variance-smooth-auc)
;;;;;;;;;;;;;;;;;;


[null, null, null, null, null, null, null, null]

In [11]:
(def accuracy-file (str "workspace/" dataset-label "/masked-AEL.csv"))
(def privacy-threshold-by-user 0.97) ;0.75, 0.8, 0.85, 0.9, 0.95, 0.97 0.98 0.985 0.99 0.995
(def final-AEL-file (str "workspace/" dataset-label "/final-ael.csv"))
(k-AEL-final-from-selected-kernel (:final-bp-file logistic-cumulative-noise-configuration) accuracy-file (:output-file-privacy-auc logistic-cumulative-noise-configuration) privacy-threshold-by-user final-AEL-file)

null

## Validate Experiments

In [18]:
;;; ;;For validating data fitting results run the experiments again with k-values received after data fitting for the privacy thresholds
(def logistic-cumulative-noise-validation
    (merge base-configuration
           {:output-file (str "workspace/" dataset-label "/validation/logistic-cumulative-output.edn")
            :output-file-privacy-auc  (str "workspace/" dataset-label "/validation/logistic-cumulative-privacy-auc.csv")
            :attck-results-file  (str "workspace/" dataset-label "/validation/attack-results.edn")
            :output-file-privacy-attacks  (str "workspace/" dataset-label "/validation/logistic-cumulative-privacy-attacks.csv")
            :original-model-description (str "workspace/" dataset-label "/validation/original-model-description.edn")
            :original-probability-file (str "workspace/" dataset-label "/validation/original-AEL.csv")
            :masked-model-file (str "workspace/" dataset-label "/validation/masked-model-description.edn")
            :masked-probability-file (str "workspace/" dataset-label "/validation/masked-AEL.csv")
            :anytime-ael-file (str "workspace/" dataset-label "/validation/anytime-AEL.csv")
            :final-bp-file (str "workspace/" dataset-label "/validation/final-breach-probabilities.csv")
            :temp-bp-file (str "workspace/" dataset-label "/validation/temp-bp.edn")
            :final-accuracy-file (str "workspace/" dataset-label "/validation/final-accuracy.csv")
            :maximum-fn-value 1  ;[1]
            :growth-rate-k-initial [0.0913 0.0738 0.1213 0.1001 0.0159] ;
                 ;k from bp- Wave 0.6749 0.0491 0.0446 0.1013 0.0626 0.0101 ,,, 0.1224 0.0101 0.5792 0.1433 0.7770 
                ;k from bp- Rational 0.1372 0.0224 0.0508 0.0947 0.0746 0.0738 ... 0.0913 0.0738 0.1213 0.1001 0.0159
            :cumulative-noise-sigmas cumulative-noise-sigmas
            :cycle-sizes cycle-sizes
            :epsilon epsilons
            :attack-strategies [:a-rp :a-rpcn :a-rpcn-1]
            :previous-AUC-file-to-update (str "workspace/" dataset-label "/validation/previous-AUC.csv")
            :previous-upper-bound-file-to-update(str "workspace/" dataset-label "/validation/previous-upper-bound.csv")
             }
    )
)
(validate-datafitting-results logistic-cumulative-noise-validation)

Starting: privacy-evaluation -pf6 -ps1.0 -cs0.001976409902290168 -tr0 -cz2000 -krc4 -krr1 -ac449 -krrp:middle -gr0.0913
Growth Rate =  0.0913
Attacks were performed
Breach probability was calculated
Written into the file
"Elapsed time: 270674.3255 msecs"
Starting: privacy-evaluation -pf6 -ps1.0 -cs0.001976409902290168 -tr0 -cz2000 -krc4 -krr1 -ac449 -krrp:middle -gr0.0738
Growth Rate =  0.0738
Attacks were performed
Breach probability was calculated
Written into the file
"Elapsed time: 274431.2991 msecs"
Starting: privacy-evaluation -pf6 -ps1.0 -cs0.001976409902290168 -tr0 -cz2000 -krc4 -krr1 -ac449 -krrp:middle -gr0.1213
Growth Rate =  0.1213
Attacks were performed
Breach probability was calculated
Written into the file
"Elapsed time: 288968.4532 msecs"
Starting: privacy-evaluation -pf6 -ps1.0 -cs0.001976409902290168 -tr0 -cz2000 -krc4 -krr1 -ac449 -krrp:middle -gr0.1001
Growth Rate =  0.1001
Attacks were performed
Breach probability was calculated
Written into the file
"Elapsed time:

[null, null, null, null, null]

In [10]:
;;Accuracy per sample for logistic noise
(def accuracy-cycles-outputfile-logistic (str "workspace/" dataset-label "/logistic-accuracy-per-cycle.edn"))
(def details-outputfile-logistic (str "workspace/" dataset-label "/logistic-accuracy-details.edn"))
(def full-cycle 1000)
(def access-result-list-logistic (nth results-cumulative-logistic 0)) ;;nth is used to access elements of lists
 (def acc-after-masking-logistic (-> access-result-list-logistic :accuracy :hdt :raw-results )) ;;getting only accuracy results
 

       ;;(accuracy-updating-cycles acc-after-masking-logistic accuracy-cycles-outputfile-logistic record-count full-cycle details-outputfile-logistic)

java.lang.RuntimeException:  Unable to resolve symbol

In [11]:
;;Average Accuracy for the attacked cycle period (Starting flat period of the logistic cycle)
(def accuracy-details-file (load-data (str "workspace/" dataset-label "/logistic-accuracy-details.edn"))) ;;reading accuracy details from previously written file
(def accuracy-for-attcked-period-file (str "workspace/" dataset-label "/logistic-accuracy-attacked-period.csv")) ;; write file of relavant accuracy details
(def accuracy-details  (-> accuracy-details-file :sample-accuracy ))  
(def full-cycle 1000)
(def flat-record-length (:flat-record-length logistic-cumulative-noise-configuration))
       (accuracy-for-flat-areas accuracy-details full-cycle flat-record-length record-count accuracy-for-attcked-period-file)


java.io.FileNotFoundException:  workspace/arem/logistic-accuracy-details.edn (No such file or directory)

In [13]:
(def get-breach-probabilty (load-data (str "workspace/" dataset-label "/logistic-attacks-per-cycle.edn"))) ;;get the final breach probability written in the file
(save-data-append-newline (str "workspace/" dataset-label "/logistic-breach-probability.csv") get-breach-probabilty) ;;append the value to another file for the operations after 100 runs

java.io.FileNotFoundException:  workspace/arem/logistic-attacks-per-cycle.edn (No such file or directory)

## Comparison of Attack Strategies

In [18]:
(doseq [epsilon epsilons cycles cycle-sizes]
    (display-html (str "<h3>Logistic Cumulative Noise With Cycles" "</h3>" "<h3>Epsilon = " epsilon "</h3>" "<h3>Cycle Size = " (* cycles 2 )"</h3>" ))
    (.display (attack-strategy-comparison-plots flat-results-cumulative-logistic :all epsilon
                                              :plot-width 400
                                              :plot-height 400
                                              :show-legend? true))
    (display-html (str "<h4>Breakdown by noise amount</h4>"))
    (.display (attack-strategy-comparison-plots flat-results-cumulative-logistic :cumulative-noise-sigma epsilon
                                                :plot-width 400
                                                :plot-height 400
                                                :show-legend? false)))

java.lang.RuntimeException:  Unable to resolve symbol

In [19]:
(let [comparison (attack-strategy-comparison flat-results-cumulative-cycles epsilons
                                  :known-record-count (apply max (:known-record-counts base-configuration)))]
    (save-data (str "workspace/" dataset-label "/cumulative-cycles-attack-strategies-comparison.edn") comparison)
    (display-table comparison))

java.lang.RuntimeException:  Unable to resolve symbol

In [20]:
(let [comparison (attack-strategy-comparison flat-results-cumulative-logistic epsilons
                                  :known-record-count (apply max (:known-record-counts base-configuration)))]
    (save-data (str "workspace/" dataset-label "/logistic-attack-strategies-comparison.edn") comparison)
    (display-table comparison))


java.lang.RuntimeException:  Unable to resolve symbol

In [21]:
;;(def best-cumulative-attack-strategy :a-rpcn-1)
(def best-cumulative-cycles-attack-strategy :a-rpcn-1)
(def best-logistic-attack-strategy :a-rpcn-1)

#'beaker_clojure_shell_23e4d944-a124-4fb2-92e2-c6028b05c2dd/best-logistic-attack-strategy

##Linear Cumulative vs.Logistic Cumulative Noise

In [22]:
(display-html (accuracy-privacy-tradeoff-legend cumulative-noise-sigmas))
nil

java.lang.RuntimeException:  Unable to resolve symbol

In [23]:
(doseq [epsilon epsilons]
    (.display (accuracy-privacy-tradeoff-comparison  results-cumulative-cycles results-cumulative-logistic ;;results-cumulative
                                                    :arf  best-cumulative-cycles-attack-strategy best-logistic-attack-strategy epsilon  ;;best-cumulative-attack-strategy
                                                    :plot-width 400
                                                    :plot-height 400)))

java.lang.RuntimeException:  Unable to resolve symbol

### Comparison of Square Distance From Origin

Performance is the sum of the squares of: (1) Prob. of e-privacy breach and (2) classification error.

In [24]:
(let [comparison (accuracy-privacy-tradeoff  results-cumulative-cycles results-cumulative-logistic ;;results-cumulative
                               :arf  best-cumulative-cycles-attack-strategy best-logistic-attack-strategy epsilons ;;best-cumulative-attack-strategy
                               :square-distance? true
                               :row-per-noise-level? true
                               :known-record-count (apply max (:known-record-counts base-configuration))
                                           )]
    (save-data (str "workspace/" dataset-label "/mask-comparison.edn") comparison)
    (display-table comparison))

java.lang.RuntimeException:  Unable to resolve symbol

## Effect of Cumulative Noise on Accuracy over Time

In [25]:
(noise-accuracy-plot  results-cumulative-cycles results-cumulative-logistic :arf ;;results-cumulative
                     :init-width 800
                     :init-height 400)

java.lang.RuntimeException:  Unable to resolve symbol

## Effect of Cumulative Noise on Privacy over Time

In [26]:
(display-masking-error-plots (->> flat-results-cumulative-cycles
                                  (filter #(= best-cumulative-cycles-attack-strategy (:strategy %)))
                                  (filter #(= (apply max cumulative-noise-sigmas) (:cumulative-noise-sigma %)))
                                  (filter #(= (apply max (:known-record-counts base-configuration)) (:known-record-count %))))
                             :cumulative-noise-sigma :known-record-count :strategy
                             :plot-width 500
                             :plot-height 500)

java.lang.RuntimeException:  Unable to resolve symbol

In [27]:
(display-masking-error-plots (->> flat-results-cumulative-logistic
                                  (filter #(= best-logistic-attack-strategy (:strategy %)))
                                  (filter #(= (apply max cumulative-noise-sigmas) (:cumulative-noise-sigma %)))
                                  (filter #(= (apply max (:known-record-counts base-configuration)) (:known-record-count %))))
                             :cumulative-noise-sigma :known-record-count :strategy
                             :plot-width 500
                             :plot-height 500)

java.lang.RuntimeException:  Unable to resolve symbol